### Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from estnltk import Text

from estnltk_patches import EntityTagger
from estnltk_patches import SyntaxTree

In [3]:
import pickle 
import random
import pandas as pd
from tqdm import tqdm
from os import listdir
from collections import Counter, defaultdict
from estnltk.wordnet import Wordnet
from estnltk.taggers import NerTagger
from estnltk.corpus_processing.parse_koondkorpus import parse_tei_corpus

### Finding sentences

In [5]:
from estnltk import Text
from estnltk.storage.postgres import PostgresStorage, create_schema


storage = PostgresStorage(host='postgres.keeleressursid.ee',
                          port=5432,
                          dbname='estonian-text-corpora',
                          user='',
                          password='',
                          schema='estonian_text_corpora',
                          role='estonian_text_corpora_read',
                          temporary=False)

INFO:storage.py:58: connecting to host: 'postgres.keeleressursid.ee', port: 5432, dbname: 'estonian-text-corpora', user: 'rasmusm'
INFO:storage.py:108: schema: 'estonian_text_corpora', temporary: False, role: 'estonian_text_corpora_read'


In [7]:
collection = storage['koondkorpus_sentences_test_5000_sg_thread']

In [8]:
storage

In [8]:
collection.selected_layers = ['v171_named_entities','v172_stanza_syntax','v172_obl_phrases']

In [ ]:
from estnltk.storage.postgres import LayerQuery
from tqdm import tqdm
texts = []
#q = LayerQuery('v172_obl_phrases')
i = 0
matching_texts = []
equal_ner_obl = []
ner_without_obl = []
ner_inside_obl = []
obl_inside_ner = []
ner_obl_loikuvad = []
partly_equal_ner_obl = []

for key, txt in tqdm(collection.select(layers=['v172_obl_phrases','v171_named_entities'])):
    texts.append(txt)
    i = i+1
    for ner in txt['v171_named_entities']:
        if ner.nertag != "LOC":
            continue
        n_match = False
        for ent in txt['v172_obl_phrases']:
            if ent.text == ner.text:
                equal_ner_obl.append((i, ner, ent))
                n_match = True
            if ent.start <= ner.end and ent.end >= ner.start:
                partly_equal_ner_obl.append((i, ner, ent))
                n_match = True
            if ner.start >= ent.start and ner.end <= ent.end:
                if ner.text == ent.text:
                    continue
                ner_inside_obl.append((i, ner, ent))
            if ent.start >= ner.start and ent.end <= ner.end:
                if ner.text == ent.text:
                    continue
                obl_inside_ner.append((i, ner, ent))
            if ner.start > ent.start and ner.end > ent.end and ner.start <= ent.end:
                ner_obl_loikuvad.append((i, ner, ent))
            if ner.start < ent.start and ner.end < ent.end and ner.end >= ent.start:
                ner_obl_loikuvad.append((i, ner, ent))
        if n_match == False:
            ner_without_obl.append((i, ner))

 28%|████████████████████▎                                                    | 44511/159745 [47:46<2:09:08, 14.87it/s]

In [36]:
ner_inside_obl

[(28,
  EnvelopingSpan(['Detroidis'], [{'nertag': 'LOC'}]),
  EnvelopingSpan(['juuniorist', ',', 'kes', 'elab', 'Eminemi', 'kodulinnas', 'Detroidis'], [{'entity_type': None, 'free_entity': None, 'is_valid': None, 'root_id': 7, 'root': <class 'estnltk_core.layer.span.Span'>}])),
 (28,
  EnvelopingSpan(['Detroidis'], [{'nertag': 'LOC'}]),
  EnvelopingSpan(['Eminemi', 'kodulinnas', 'Detroidis'], [{'entity_type': None, 'free_entity': None, 'is_valid': None, 'root_id': 12, 'root': <class 'estnltk_core.layer.span.Span'>}])),
 (52,
  EnvelopingSpan(['Hollywoodi'], [{'nertag': 'LOC'}]),
  EnvelopingSpan(['Hollywoodi', 'või', 'New', 'Yorki'], [{'entity_type': None, 'free_entity': None, 'is_valid': None, 'root_id': 11, 'root': <class 'estnltk_core.layer.span.Span'>}])),
 (52,
  EnvelopingSpan(['New', 'Yorki'], [{'nertag': 'LOC'}]),
  EnvelopingSpan(['Hollywoodi', 'või', 'New', 'Yorki'], [{'entity_type': None, 'free_entity': None, 'is_valid': None, 'root_id': 11, 'root': <class 'estnltk_core.laye

In [25]:
len(collection)*0.12

19169.399999999998

In [19]:
len(matching_texts)

6

In [20]:
matching_texts[0]

(Text(text='Aasta on 1995 , Jimmy on koolist välja visatud vaene valgenahaline poiss , kes elab koos alkohoolikust emaga ( Kim Basinger ) Kaheksa Miili nimelise tee ääres treileris .'),
 16,
 21)

In [22]:
matching_texts[0][0].v172_obl_phrases

layer name,attributes,parent,enveloping,ambiguous,span count
v172_obl_phrases,"entity_type, free_entity, is_valid, root_id, root",None,v172_stanza_syntax,False,6
text,entity_type,free_entity,is_valid,root_id,root
['Jimmy'],None,None,None,5,"Span('Jimmy', [{'id': 5, 'lemma': 'Jimmy', 'upostag': 'S', 'xpostag': 'S', 'feat ..., type: <class 'estnltk_core.layer.span.Span'>"
['koolist'],None,None,None,7,"Span('koolist', [{'id': 7, 'lemma': 'kool', 'upostag': 'S', 'xpostag': 'S', 'fea ..., type: <class 'estnltk_core.layer.span.Span'>"
"['koos', 'alkohoolikust']",None,None,None,17,"Span('alkohoolikust', [{'id': 17, 'lemma': 'alkohoolik', 'upostag': 'S', 'xposta ..., type: <class 'estnltk_core.layer.span.Span'>"
"['emaga', '(', 'Kim', 'Basinger', ')']",None,None,None,18,"Span('emaga', [{'id': 18, 'lemma': 'ema', 'upostag': 'S', 'xpostag': 'S', 'feats ..., type: <class 'estnltk_core.layer.span.Span'>"
"['Kaheksa', 'Miili', 'nimelise', 'tee', 'ääres']",None,None,None,26,"Span('tee', [{'id': 26, 'lemma': 'tee', 'upostag': 'S', 'xpostag': 'S', 'feats': ..., type: <class 'estnltk_core.layer.span.Span'>"
['treileris'],None,None,None,28,"Span('treileris', [{'id': 28, 'lemma': 'treiler', 'upostag': 'S', 'xpostag': 'S' ..., type: <class 'estnltk_core.layer.span.Span'>"


In [16]:
texts[0]

Text(text='Välismaa leheküljed toimetanud Aive Lauriste')

In [9]:
with open('sampled_obl_texts.pickle','wb') as f:
    pickle.dump(texts,f)

MemoryError: 

In [15]:
len(collection)

159745

In [10]:
len(texts)

159745

In [8]:
# Second storage to keep the temporary lists used for sampling
localstorage = PostgresStorage(host='localhost',
                          port=5432,
                          dbname='ner_test',
                          user='',
                          password='',
                          pgpass_file='~/.pgpass',
                          schema='my_schema',
                          role=None,
                          temporary=False)

INFO:storage.py:58: connecting to host: 'localhost', port: 5432, dbname: 'ner_test', user: 'postgres'
INFO:storage.py:108: schema: 'my_schema', temporary: False, role: 'postgres'


In [16]:
collection[3].v172_obl_phrases

layer name,attributes,parent,enveloping,ambiguous,span count
v172_obl_phrases,"entity_type, free_entity, is_valid, root_id, root",None,v172_stanza_syntax,False,2
text,entity_type,free_entity,is_valid,root_id,root
"['Kaheksakümnendate', 'aastate', 'lõpus']",None,None,None,3,"Span('lõpus', [{'id': 3, 'lemma': 'lõpp', 'upostag': 'S', 'xpostag': 'S', 'feats ..., type: <class 'estnltk_core.layer.span.Span'>"
['1.'],None,None,None,6,"Span('1.', [{'id': 6, 'lemma': '1.', 'upostag': 'N', 'xpostag': 'N', 'feats': {' ..., type: <class 'estnltk_core.layer.span.Span'>"


In [ ]:
sampler = SpanSampler(storage=localstorage,collection=collection, layer='v172_obl_phrases',attribute='lemma')

### OBL layer

In [86]:
obl_extractor = EntityTagger(deprel='obl', output_layer='entities')

[autoreload of estnltk.taggers.system.rule_taggers.helper_methods.helper_methods failed: Traceback (most recent call last):
  File "C:\Users\rasmu\anaconda3\envs\estnltk\lib\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\rasmu\anaconda3\envs\estnltk\lib\site-packages\IPython\extensions\autoreload.py", line 394, in superreload
    module = reload(module)
  File "C:\Users\rasmu\anaconda3\envs\estnltk\lib\imp.py", line 315, in reload
    return importlib.reload(module)
  File "C:\Users\rasmu\anaconda3\envs\estnltk\lib\importlib\__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 674, in exec_module
  File "<frozen importlib._bootstrap_external>", line 781, in get_code
  File "<frozen importlib._bootstrap_external>", line 741, in source_to_code
  File "<frozen importlib._bootstrap

In [87]:
from estnltk.web_taggers import StanzaSyntaxWebTagger

stanza = StanzaSyntaxWebTagger(url='https://api.tartunlp.ai/estnltk/tagger/stanza_syntax')

In [88]:
for sent in texts:
    sent.tag_layer('morph_extended')
    stanza(sent)
    obl_extractor(sent)

### NER and OBL comparison

In [4]:
#with open("original_sentences_with_analysis.pickle", "rb") as f:
#    sentences = pickle.load(f)

In [116]:
with open("ner_obl_samples.pickle","wb") as f:
    pickle.dump(texts, f)

In [94]:
equal_ner_obl = []
partly_equal_ner_obl = []
ner_without_obl = []

for i, sentence in enumerate(texts):
    try:
        ents = sentence.entities
        nerlayer = sentence.v171_named_entities
    
        for n in nerlayer:
            if n.nertag != "LOC":
                continue
            n_match = False
            for ent in ents:
                if ent.text == n.text:
                    equal_ner_obl.append((i, n, ent))
                    n_match = True
                if ent[0].start <= n[-1].end and ent[-1].end >= n[0].start:
                    partly_equal_ner_obl.append((i, n, ent))
                    n_match = True
            if n_match == False:
                ner_without_obl.append((i, n))
    except:
        print(i)
        continue

In [95]:
len(equal_ner_obl)

304

In [96]:
equal_ner_obl[0]

(1,
 EnvelopingSpan(['Eestisse'], [{'nertag': 'LOC'}]),
 EnvelopingSpan(['Eestisse'], [{'entity_type': None, 'free_entity': None, 'is_valid': None, 'root': <class 'estnltk_core.layer.span.Span'>}]))

In [97]:
len(partly_equal_ner_obl)

881

In [98]:
partly_equal_ner_obl[0]

(0,
 EnvelopingSpan(['Palestiina'], [{'nertag': 'LOC'}]),
 EnvelopingSpan(['Palestiina', 'omavalitsuselt'], [{'entity_type': None, 'free_entity': None, 'is_valid': None, 'root': <class 'estnltk_core.layer.span.Span'>}]))

In [99]:
partly_equal_ner_obl[:10]

[(0,
  EnvelopingSpan(['Palestiina'], [{'nertag': 'LOC'}]),
  EnvelopingSpan(['Palestiina', 'omavalitsuselt'], [{'entity_type': None, 'free_entity': None, 'is_valid': None, 'root': <class 'estnltk_core.layer.span.Span'>}])),
 (0,
  EnvelopingSpan(['Iisraelil'], [{'nertag': 'LOC'}]),
  EnvelopingSpan(['ka', 'Iisraelil'], [{'entity_type': None, 'free_entity': None, 'is_valid': None, 'root': <class 'estnltk_core.layer.span.Span'>}])),
 (1,
  EnvelopingSpan(['Eestisse'], [{'nertag': 'LOC'}]),
  EnvelopingSpan(['Eestisse'], [{'entity_type': None, 'free_entity': None, 'is_valid': None, 'root': <class 'estnltk_core.layer.span.Span'>}])),
 (2,
  EnvelopingSpan(['Istanbulis'], [{'nertag': 'LOC'}]),
  EnvelopingSpan(['Istanbulis'], [{'entity_type': None, 'free_entity': None, 'is_valid': None, 'root': <class 'estnltk_core.layer.span.Span'>}])),
 (7,
  EnvelopingSpan(['Itaalias'], [{'nertag': 'LOC'}]),
  EnvelopingSpan(['Itaalias'], [{'entity_type': None, 'free_entity': None, 'is_valid': None, 'ro

In [100]:
len(ner_without_obl)

783

In [101]:
ner_without_obl[0]

(2, EnvelopingSpan(['Platini'], [{'nertag': 'LOC'}]))

In [105]:
ner_inside_obl = []

for i, sentence in enumerate(texts):
    try:
        ents = sentence.entities
        nerlayer = sentence.v171_named_entities
        
        if len(ents) == 0:
            continue
    
        for n in nerlayer:
            if n.nertag != "LOC":
                continue
            for ent in ents:
                if n[0].start >= ent[0].start and n[-1].end <= ent[-1].end:
                    if n.text == ent.text:
                        continue
                    ner_inside_obl.append((i, n, ent))
    except Exception as e:
        print(i, e)
        continue

In [106]:
len(ner_inside_obl)

578

In [107]:
ner_inside_obl[:5]

[(0,
  EnvelopingSpan(['Palestiina'], [{'nertag': 'LOC'}]),
  EnvelopingSpan(['Palestiina', 'omavalitsuselt'], [{'entity_type': None, 'free_entity': None, 'is_valid': None, 'root': <class 'estnltk_core.layer.span.Span'>}])),
 (0,
  EnvelopingSpan(['Iisraelil'], [{'nertag': 'LOC'}]),
  EnvelopingSpan(['ka', 'Iisraelil'], [{'entity_type': None, 'free_entity': None, 'is_valid': None, 'root': <class 'estnltk_core.layer.span.Span'>}])),
 (8,
  EnvelopingSpan(['Eesti'], [{'nertag': 'LOC'}]),
  EnvelopingSpan(['Eesti', 'maksupoliitikat', 'eeskujuks', 'toonud', 'Kallase', 'väitel'], [{'entity_type': None, 'free_entity': None, 'is_valid': None, 'root': <class 'estnltk_core.layer.span.Span'>}])),
 (8,
  EnvelopingSpan(['Euroopa'], [{'nertag': 'LOC'}]),
  EnvelopingSpan(['Euroopa', 'valimistel'], [{'entity_type': None, 'free_entity': None, 'is_valid': None, 'root': <class 'estnltk_core.layer.span.Span'>}])),
 (10,
  EnvelopingSpan(['Eesti'], [{'nertag': 'LOC'}]),
  EnvelopingSpan(['temast', 'kui'

In [91]:
ner_inside_obl[5:10]

[(221,
  EnvelopingSpan(['Eesti'], [{'nertag': 'LOC'}]),
  EnvelopingSpan(['Eesti', 'kindlustatute', 'ravimise', 'eest'], [{'entity_type': None, 'free_entity': None, 'is_valid': None, 'root': <class 'estnltk_core.layer.span.Span'>}])),
 (221,
  EnvelopingSpan(['Eestis'], [{'nertag': 'LOC'}]),
  EnvelopingSpan(['vaid', 'Eestis'], [{'entity_type': None, 'free_entity': None, 'is_valid': None, 'root': <class 'estnltk_core.layer.span.Span'>}])),
 (224,
  EnvelopingSpan(['Soome'], [{'nertag': 'LOC'}]),
  EnvelopingSpan(['Soome', ',', 'Rootsi', ',', 'Läti', 'ja', 'Leeduga'], [{'entity_type': None, 'free_entity': None, 'is_valid': None, 'root': <class 'estnltk_core.layer.span.Span'>}])),
 (224,
  EnvelopingSpan(['Rootsi'], [{'nertag': 'LOC'}]),
  EnvelopingSpan(['Soome', ',', 'Rootsi', ',', 'Läti', 'ja', 'Leeduga'], [{'entity_type': None, 'free_entity': None, 'is_valid': None, 'root': <class 'estnltk_core.layer.span.Span'>}])),
 (224,
  EnvelopingSpan(['Läti'], [{'nertag': 'LOC'}]),
  Envelopi

In [108]:
obl_inside_ner = []

for i, sentence in enumerate(texts):
    try:
        ents = sentence.entities
        nerlayer = sentence.v171_named_entities
        
        if len(ents) == 0:
            continue
            
        for ent in ents:
            for n in nerlayer:
                if n.nertag != "LOC":
                    continue
                if ent[0].start >= n[0].start and ent[-1].end <= n[-1].end:
                    if n.text == ent.text:
                        continue
                    obl_inside_ner.append((i, n, ent))
    except Exception as e:
        print(i, e)
        continue

In [109]:
len(obl_inside_ner)

2

In [110]:
obl_inside_ner[:5]

[(426,
  EnvelopingSpan(['Ungaris', 'Balatonil'], [{'nertag': 'LOC'}]),
  EnvelopingSpan(['Ungaris'], [{'entity_type': None, 'free_entity': None, 'is_valid': None, 'root': <class 'estnltk_core.layer.span.Span'>}])),
 (426,
  EnvelopingSpan(['Ungaris', 'Balatonil'], [{'nertag': 'LOC'}]),
  EnvelopingSpan(['Balatonil'], [{'entity_type': None, 'free_entity': None, 'is_valid': None, 'root': <class 'estnltk_core.layer.span.Span'>}]))]

In [111]:
ner_obl_loikuvad = []

for i, sentence in enumerate(texts):
    try:
        ents = sentence.entities
        nerlayer = sentence.v171_named_entities
        
        if len(ents) == 0:
            continue
    
        for n in nerlayer:
            if n.nertag != "LOC":
                continue
            for ent in ents:
                if n[0].start > ent[0].start and n[-1].end > ent[-1].end and n[0].start <= ent[-1].end:
                    ner_obl_loikuvad.append((i, n, ent))
                if n[0].start < ent[0].start and n[-1].end < ent[-1].end and n[-1].end >= ent[0].start:
                    ner_obl_loikuvad.append((i, n, ent))
    except Exception as e:
        print(i, e)
        continue

In [112]:
len(ner_obl_loikuvad)

1

In [113]:
ner_obl_loikuvad[:5]

[(439,
  EnvelopingSpan(['Austraaliasse', 'Landsbergise'], [{'nertag': 'LOC'}]),
  EnvelopingSpan(['Landsbergise', 'pressisekretäri', 'sõnul'], [{'entity_type': None, 'free_entity': None, 'is_valid': None, 'root': <class 'estnltk_core.layer.span.Span'>}]))]

In [99]:
ner_obl_eq_200 = random.sample(equal_ner_obl, 200)
ner_in_obl_200 = random.sample(ner_inside_obl, 200)
## obl_in_ner_200 = random.sample(obl_inside_ner, 200)

In [101]:
ner_obl_eq_200[0]

(8960,
 EnvelopingSpan(['Eestis'], [{'nertag': 'LOC'}]),
 EnvelopingSpan(['Eestis'], [{'entity_type': None, 'free_entity': None, 'is_valid': None, 'root': <class 'estnltk_core.layer.span.Span'>}]))

In [102]:
sentences[ner_obl_eq_200[0][0]].text

'Patsientide esindajate veendumuse kohaselt seevastu tuleks just patsiendiseaduses sätestada , et tervishoiuteenused peavad olema pidevalt kättesaadavad kõigile Eestis alaliselt elavatele isikutele .'

In [103]:
" ".join(ner_obl_eq_200[0][1].text)

'Eestis'

In [104]:
" ".join(ner_obl_eq_200[0][2].text)

'Eestis'

In [105]:
" ".join(ner_obl_loikuvad[0][1].text)

'Ridala valla'

In [106]:
ner_obl_eq_200_proc = ["\t".join([sentences[i].text, " ".join(ner.text), " ".join(ent.text)]) for i, ner, ent in ner_obl_eq_200]

In [107]:
ner_obl_eq_200_proc[:3]

['Patsientide esindajate veendumuse kohaselt seevastu tuleks just patsiendiseaduses sätestada , et tervishoiuteenused peavad olema pidevalt kättesaadavad kõigile Eestis alaliselt elavatele isikutele .\tEestis\tEestis',
 'Tiit Madisson elab Lihula vallas Kirikukülas viiendat aastat ning kasvatab Väinamere pärandkoosluste kaitse ühingu liikmena Matsalu looduskaitsealal šoti mägiveiseid , hereforde ja 40 lammast .\tKirikukülas\tKirikukülas',
 'Mõned Naeri sõbrad ja fännid kaugelt maalt tahavad puhkuse ajaks Eestisse tulla .\tEestisse\tEestisse']

In [108]:
ner_in_obl_200_proc = ["\t".join([sentences[i].text, " ".join(ner.text), " ".join(ent.text)]) for i, ner, ent in ner_in_obl_200]
obl_in_ner_proc = ["\t".join([sentences[i].text, " ".join(ner.text), " ".join(ent.text)]) for i, ner, ent in obl_inside_ner]
ner_obl_loikuvad_proc = ["\t".join([sentences[i].text, " ".join(ner.text), " ".join(ent.text)]) for i, ner, ent in ner_obl_loikuvad]

In [109]:
ner_obl_loikuvad_proc[0]

'Taani toidu- , põllumajandus- ja kalandusminister Ritt Bjerregaard käis Ridala valla mahetalunike põldudel , et veenduda , kas Taani-Eesti ühisprojekti raha on kasutatud asja eest .\tRidala valla\tvalla mahetalunike põldudel'

In [110]:
with open("ner_vs_obl/ner_eq_obl.txt", "w") as f:
    for line in ner_obl_eq_200_proc:
        f.write(f"{line}\n")

In [111]:
with open("ner_vs_obl/ner_in_obl.txt", "w") as f:
    for line in ner_in_obl_200_proc:
        f.write(f"{line}\n")

In [112]:
with open("ner_vs_obl/obl_in_ner.txt", "w") as f:
    for line in obl_in_ner_proc:
        f.write(f"{line}\n")

In [113]:
with open("ner_vs_obl/ner_obl_loikuvad.txt", "w") as f:
    for line in ner_obl_loikuvad_proc:
        f.write(f"{line}\n")

### Labelstudio conversion

In [123]:
from estnltk.converters.label_studio.label_studio import LabelStudioExporter

In [127]:
exporter = LabelStudioExporter("nerobltest.json",['entities','v171_named_entities'],checkbox=True)

In [128]:
print(exporter.labeling_interface)


        <View>
            <Labels name="label" toName="text">
	<Label value="entities" background="#71E722"/> 
	<Label value="v171_named_entities" background="#B47DEF"/> 

            </Labels>
        <Text name="text" value="$text"/>
            <Header value="Are the annotations correct?"/>
                <Choices name="review" toName="text">
                    <Choice value="yes"/>
                    <Choice value="no"/>
                </Choices>
            </View>


In [126]:
exporter.convert(texts,append=False)